# **NER 사전 만들기**
https://pypi.org/project/mdict-utils/
1. **MDX 사전파일을** 활용하여 객체에 구분하기

## **1 Sqlite3 데이터 불러오기**
sqlite3 데이터 불러오기

In [18]:
# import sqlite3
# import pandas as pd
# conn = sqlite3.connect('backup/kordict.db')
# resp = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
# tableNames = [name[0]  for name in resp]
# tableNames

In [19]:
# %%time
# df = pd.read_sql_query("SELECT * FROM {}".format(tableNames[1]), conn)
# itemIndex = df.entry.values.tolist()
# itemData  = df.paraphrase.values.tolist()
# print(len(itemIndex), itemIndex[:10], df.shape)
# df.head()

## **2 자료 전처리**
1. regex 를 활용하여 가치있는 **명사내용** 추출하기
1. "「1」" 내용은 동일한 개념의 단어를 정리한 것
1. sentence 별로 내용의 확인 및 구분이 필요

```html
<font color=crimson size=+2><b>대상행동</b></font>
<br><br>
<font COLOR="#00008B"><b>대상^행동(代償行動)</b></font>
『심리』<br> 자기가 요구하는 바를 얻지 못할 때 그와 비슷한 다른 대상으로 만족을 채우려는 행동. ≒보상 작용
<br>「1」ㆍ보상 행동.\r\n'
 ```

In [20]:
# # 색인정보 : 51만개의 단어사전 데이터
# itemIndex = df.entry.values.tolist()
# itemData  = df.paraphrase.values.tolist()
# len(itemIndex), itemIndex[:10]

In [1]:
import pickle
with open('data/nerDict.pk', 'rb') as handle:
    nerDict = pickle.load(handle)
itemIndex = list(nerDict.Text)
itemData  = list(nerDict.Data)
len(itemIndex), len(itemData)

388091

## **3 HTML 태그 제거 및 본문선별**
1. **/font** 태그가 제목 이외에, 세부내용에도 포함됩니다.
1. 때문에 이를 무조건 적용하기엔 문제가 있어서 Tag 구조를 명확하게 파악하기
1. 우선 구조에 대한 형태를 몇가지로 확인한 뒤 작업을 진행하기

```python
'<title.*?>(.+?)</title>'  # 특정태그
"<[^>]+>|[^<]+"            # html 태그 내부의 한글 추출
'<.*?>'                    # 모든 태그
```

In [29]:
search_query = "대리인"
num = [no  for no, _ in enumerate(itemIndex)  if _ == search_query][0]
num

79582

In [ ]:
tempXml = itemData[num].lower()
from bs4 import BeautifulSoup
html_source = BeautifulSoup(tempXml, 'html.parser').prettify()
html_source # print(html_source)

In [ ]:
import re
# regex_html = '<.*>'                 # html 태그내용만 추출 (탐욕적 수량자)
regex_html = '<.*?>'                 # html 태그내용만 추출 (게으른 수량자)
regex_text = "<[^>]+>|[^<]+"         # html 태그 내부의 한글 추출
regex_font = '<font.*?>(.+?)</font>' # html 추출 : font 태그 내용만 추출

# html 태그의 추출
f_html = re.findall(regex_html, tempXml) # HTML 태그내용
", ".join(f_html)

In [ ]:
# 특정 font 태그 내부의 text 를 추출
f_font = re.findall(regex_font, tempXml)
["<font>{}</font>".format(_) for _ in f_font]
[_ for _ in f_font]

In [ ]:
# f_font 의 태그를 제거한 나머지
f_txt = [re.sub(regex_html,"", _)  for _ in f_font] 
f_txt

In [ ]:
f_full = re.findall(regex_text, tempXml) 
", ".join(f_full)

In [ ]:
# 「1」', '「2」 개념설명 1, 2
# ¶  문장의 예시
[_  for _ in f_full  if _ not in f_html]

In [ ]:
# font 태그 내용을 제거한 나머지
[_  for _ in f_full  if _ not in f_html  if _ not in f_txt]

In [ ]:
%%time
result = []
for _ in itemData:
    result += re.findall(regex_html, _) # HTML 태그내용
result      = list(set(result))
result_tags = []         # 결과 중 사진포함 내용을 제거한 나머지
regex_img   = '<img.*?>' # html 추출 : font 태그 내용만 추출
for _ in result:
    _ = re.sub(regex_img, "", _)
    if len(_) > 2:
        result_tags.append(_.lower())
len(set(result_tags)), sorted(set(result_tags))

## **4 분류 작업용 함수 만들기**
3개의 분류를 활용하여 List 만들기, 이후 대표적인 대상만 남기고 줄이기
1. 타이들
1. 대분류
1. 소분류 1, 2

In [ ]:
search_query = "대리인"
num     = [no  for no, _ in enumerate(itemIndex)  if _ == search_query][0]
tempXml = itemData[num].lower()
tempXml

In [ ]:
import re
regex_html = '<.*?>'                 # html 태그내용만 추출
regex_text = "<[^>]+>|[^<]+"         # html 태그 내부의 한글 추출
regex_font = '<font.*?>(.+?)</font>' # html 추출 : font 태그 내용만 추출
f_full = re.findall(regex_text, tempXml) 
f_html = re.findall(regex_html, tempXml) # HTML 태그내용
f_test = [_  for _ in f_full  if _ not in f_html]
f_test

In [ ]:
# (?<=\$)
test  = "「명사」「1」「동-사」"
t_txt = re.compile(r"「[·가-힣]+」")
t_txt = re.compile(r"「([·가-힣]+)」")
t_num = re.compile(r"「\d+」")
t_box = re.compile(r"「.*?」")
# t_box = re.compile(r"(?<=\「).*?(?<=\」)$")
# t_num.findall(test)
# t_txt.findall(test)
t_box.findall(test)

In [ ]:
search_query = "대리인"
num     = [no  for no, _ in enumerate(itemIndex)  if _ == search_query][0]
tempXml = itemData[num].lower()
tempXml

In [ ]:
result, result_len = [], []
for _ in itemData:
    temp = t_txt.findall(_)
    temp_num = len(temp)
    result_len.append(temp_num)
    if temp_num > 1:
        result += temp
result =list(set(result))
len(result)

## **4 분류 작업용 함수 만들기 2**
1. "「.*?」" 를 기준으로 객체들을 구분 분류하기
1. 추후 숫자와 개념어를 구분하여 분류 재정의 하기

In [ ]:
# 텍스트 인덱스값 찾기
search_query = "가결"
search_query = "대리인"
search_query = "서버"
num     = [no  for no, _ in enumerate(itemIndex)  if _ == search_query][0]
tempXml = itemData[num].lower()
tempXml

In [ ]:
import re
regex_html = '<.*?>'                 # html 태그내용만 추출
regex_text = "<[^>]+>|[^<]+"         # html 태그 내부의 한글 추출
f_txt      = [re.sub(regex_html,"", _)  for _ in f_font] 
f_full     = re.findall(regex_text, tempXml)
t_temp     = [_  for _ in f_full  if _ not in f_html]
t_temp

In [ ]:
result = []
for _ in itemData:
    temp = ''
    tok  = re.compile(r"(「.*」)")
    for t in tok.findall(_):
        if len(t) < 10:
            t = t.replace("「","")
            temp += ","+ t.replace("」","")
    result.append(temp)
len(result)

In [ ]:
result_html = " ".join(list(set(result)))

In [ ]:
test = "<img src='hc513sy.jpg' width='300'> 테스트 추출"

## **3 HTML 태그 제거 및 본문선별**
1. **/font** 태그가 제목 이외에, 세부내용에도 포함됩니다.
1. 때문에 이를 무조건 적용하기엔 문제가 있어서 Tag 구조를 명확하게 파악하기
1. 우선 구조에 대한 형태를 몇가지로 확인한 뒤 작업을 진행하기

```python
'<title.*?>(.+?)</title>'  # 특정태그
'<.*?>' # 모든 태그
```

In [ ]:
tokenizer = re.compile("<*h4[^>]*>(.*?)<\s*/\s*h4>/g")

In [ ]:
tokenizer.findall('<h4 class="sds">And more ...</h4>')

In [ ]:
re.findall

In [ ]:
regex = re.compile("/<(.|\n)*?>/")

In [ ]:
regex.finditer("<p>test</p>")

In [ ]:
idxNo = 12004
itemTemp = itemData[idxNo].lower()
toknizer = re.compile('<.*?>')
toknizer_class = re.compile('class="[a-zA-Z0-9:;\.\s\(\)\-\,]*"')
# re.sub(toknizer, '', itemTemp)
re.findall(toknizer, itemTemp)[6]

## **3 Regex를 활용하여 구분하기**
본문에 "\<br\>" 내용이 많아서 "//text()" 를 사용하면 컬럼갯수가 꼬여버린다
regex, lxml 을 사용하면 좋겠지만 경우의 수가 많아서 모두 처리하지 않으면 안됨..

In [ ]:
data = itemData[2945]
data = data.lower()  # html Tag 전처리
data

In [ ]:
import re
re.findall(r'<font color=crimson (.*?)</font>', data, re.DOTALL)

In [ ]:
import re # 강결합으로 정치큰 부분 제거하기
re.sub('<br><font color=.*</font>', "",data)

In [ ]:
_ = re.sub('<font .*</font><br>', "",data).split("<br>")
_[0], _[1].replace("\r\n", "")

In [ ]:
import re
from lxml.html import fromstring
def exportMdx(data):
    data = data.lower()  # html Tag 전처리
    _ = re.sub('<font .*</font><br>', "",data).split("<br>")
    return fromstring(data).xpath('//font//text()'), (_[0], _[1].replace("\r\n", ""))